<a href="https://colab.research.google.com/github/3srava0/assignment-3/blob/main/streamlit_app_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Day 7: Streamlit App Development
# Real Estate Investment Advisor System
print("="*60)
print("STREAMLIT APP DEVELOPMENT")
print("="*60)

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Create app directory
import os

app_dir = '/content/drive/MyDrive/assignment-3/streamlit_app'
os.makedirs(app_dir, exist_ok=True)

print(f"\n✅ App directory created: {app_dir}")


STREAMLIT APP DEVELOPMENT
Mounted at /content/drive

✅ App directory created: /content/drive/MyDrive/assignment-3/streamlit_app


In [2]:
# Install Streamlit and dependencies
!pip install streamlit -q
!pip install pyngrok -q

print("✅ Streamlit and dependencies installed!")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 104.8 MB/s eta 0:00:00
✅ Streamlit and dependencies installed!


In [3]:
# Create main Streamlit app
app_code = '''
import streamlit as st
import pandas as pd
import numpy as np
import pickle
import json
from pathlib import Path

# Page configuration
st.set_page_config(
    page_title="Real Estate Investment Advisor",
    page_icon="🏠",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Custom CSS
st.markdown("""
    <style>
    .main-header {
        font-size: 42px;
        font-weight: bold;
        color: #1f77b4;
        text-align: center;
        padding: 20px;
    }
    .prediction-box {
        padding: 20px;
        border-radius: 10px;
        margin: 10px 0;
    }
    .good-investment {
        background-color: #d4edda;
        border: 2px solid #28a745;
    }
    .bad-investment {
        background-color: #f8d7da;
        border: 2px solid #dc3545;
    }
    .metric-card {
        background-color: #f0f2f6;
        padding: 15px;
        border-radius: 8px;
        margin: 5px;
    }
    </style>
""", unsafe_allow_html=True)

# Load models
@st.cache_resource
def load_models():
    models_dir = Path("models")

    with open(models_dir / "final_tuned_classification_model.pkl", "rb") as f:
        clf_model = pickle.load(f)

    with open(models_dir / "final_tuned_regression_model.pkl", "rb") as f:
        reg_model = pickle.load(f)

    with open(models_dir / "final_scaler.pkl", "rb") as f:
        scaler = pickle.load(f)

    with open(models_dir / "label_encoders.pkl", "rb") as f:
        encoders = pickle.load(f)

    with open(models_dir / "final_model_metrics.json", "r") as f:
        metrics = json.load(f)

    return clf_model, reg_model, scaler, encoders, metrics

# Load data for reference
@st.cache_data
def load_reference_data():
    data = pd.read_csv("data/data_with_targets.csv")
    return data

# Main app
def main():
    # Header
    st.markdown('<p class="main-header">🏠 Real Estate Investment Advisor</p>', unsafe_allow_html=True)
    st.markdown("### Predict Property Profitability & Future Value")
    st.markdown("---")

    # Load models and data
    try:
        clf_model, reg_model, scaler, encoders, metrics = load_models()
        reference_data = load_reference_data()
    except Exception as e:
        st.error(f"Error loading models: {e}")
        st.stop()

    # Sidebar - Model Performance
    with st.sidebar:
        st.header("📊 Model Performance")

        st.subheader("Classification Model")
        st.metric("ROC-AUC Score", f"{metrics[\\'classification\\'][\\'test_roc_auc\\']:.4f}")
        st.metric("Accuracy", f"{metrics[\\'classification\\'][\\'test_accuracy\\']:.4f}")

        st.subheader("Regression Model")
        st.metric("R² Score", f"{metrics[\\'regression\\'][\\'test_r2\\']:.4f}")
        st.metric("RMSE", f"{metrics[\\'regression\\'][\\'test_rmse\\']:.2f} Lakhs")

        st.markdown("---")
        st.info("💡 Enter property details to get investment advice!")

    # Main content - Tabs
    tab1, tab2, tab3 = st.tabs(["🔮 Prediction", "📈 Market Insights", "ℹ️ About"])

    with tab1:
        st.header("Property Details Input")

        # Create form
        col1, col2, col3 = st.columns(3)

        with col1:
            st.subheader("📍 Location")
            state = st.selectbox("State", sorted(reference_data[\\'State\\'].unique()))
            city = st.selectbox("City", sorted(reference_data[reference_data[\\'State\\']==state][\\'City\\'].unique()))
            locality = st.text_input("Locality", "Locality_1")

            st.subheader("🏗️ Property Type")
            property_type = st.selectbox("Type", ["Apartment", "Villa", "Independent House", "Penthouse"])
            bhk = st.number_input("BHK", min_value=1, max_value=10, value=2)
            size_sqft = st.number_input("Size (Sq Ft)", min_value=300, max_value=10000, value=1500)

        with col2:
            st.subheader("💰 Pricing")
            current_price = st.number_input("Current Price (Lakhs)", min_value=10.0, max_value=1000.0, value=100.0, step=5.0)
            price_per_sqft = current_price * 100000 / size_sqft

            st.subheader("🏢 Building Details")
            year_built = st.number_input("Year Built", min_value=1980, max_value=2026, value=2015)
            floor_no = st.number_input("Floor Number", min_value=0, max_value=50, value=3)
            total_floors = st.number_input("Total Floors", min_value=1, max_value=50, value=10)
            furnished_status = st.selectbox("Furnished Status", ["Unfurnished", "Semi-Furnished", "Fully-Furnished"])

        with col3:
            st.subheader("🎯 Amenities & Features")
            nearby_schools = st.slider("Nearby Schools", 0, 20, 5)
            nearby_hospitals = st.slider("Nearby Hospitals", 0, 20, 3)
            transport_access = st.selectbox("Public Transport", ["Low", "Medium", "High"])

            parking = st.selectbox("Parking Space", ["Yes", "No"])
            security = st.selectbox("Security", ["Yes", "No"])
            amenities = st.multiselect("Amenities", ["Gym", "Pool", "Playground", "Clubhouse", "Garden"])
            facing = st.selectbox("Facing", ["North", "South", "East", "West"])
            owner_type = st.selectbox("Owner Type", ["Owner", "Builder", "Broker"])

        # Predict button
        if st.button("🔮 Predict Investment Quality & Future Price", use_container_width=True):

            # Prepare input data
            age_of_property = 2026 - year_built
            amenities_str = ", ".join(amenities) if amenities else "None"

            input_data = {
                \\'State\\': state,
                \\'City\\': city,
                \\'Locality\\': locality,
                \\'Property_Type\\': property_type,
                \\'BHK\\': bhk,
                \\'Size_in_SqFt\\': size_sqft,
                \\'Price_per_SqFt\\': price_per_sqft,
                \\'Year_Built\\': year_built,
                \\'Furnished_Status\\': furnished_status,
                \\'Floor_No\\': floor_no,
                \\'Total_Floors\\': total_floors,
                \\'Age_of_Property\\': age_of_property,
                \\'Nearby_Schools\\': nearby_schools,
                \\'Nearby_Hospitals\\': nearby_hospitals,
                \\'Public_Transport_Accessibility\\': transport_access,
                \\'Parking_Space\\': parking,
                \\'Security\\': security,
                \\'Amenities\\': amenities_str,
                \\'Facing\\': facing,
                \\'Owner_Type\\': owner_type,
                \\'Availability_Status\\': "Available"
            }

            input_df = pd.DataFrame([input_data])

            # Encode categorical variables
            for col in input_df.select_dtypes(include=[\\'object\\']).columns:
                if col in encoders:
                    try:
                        input_df[col] = encoders[col].transform(input_df[col].astype(str))
                    except:
                        input_df[col] = 0  # Unknown category
                else:
                    input_df[col] = 0

            # Make predictions
            try:
                # Classification
                clf_pred = clf_model.predict(input_df)[0]
                clf_proba = clf_model.predict_proba(input_df)[0]

                # Regression
                reg_pred = reg_model.predict(input_df)[0]

                # Display results
                st.markdown("---")
                st.header("🎯 Prediction Results")

                # Classification result
                col_a, col_b = st.columns(2)

                with col_a:
                    if clf_pred == 1:
                        st.markdown(
                            f"""
                            <div class="prediction-box good-investment">
                                <h2>✅ GOOD INVESTMENT</h2>
                                <p style="font-size:20px">Confidence: {clf_proba[1]*100:.1f}%</p>
                            </div>
                            """,
                            unsafe_allow_html=True
                        )
                    else:
                        st.markdown(
                            f"""
                            <div class="prediction-box bad-investment">
                                <h2>❌ RISKY INVESTMENT</h2>
                                <p style="font-size:20px">Confidence: {clf_proba[0]*100:.1f}%</p>
                            </div>
                            """,
                            unsafe_allow_html=True
                        )

                with col_b:
                    st.markdown(
                        f"""
                        <div class="metric-card">
                            <h3>📈 Predicted Price (5 Years)</h3>
                            <h1 style="color:#1f77b4">₹{reg_pred:.2f} Lakhs</h1>
                            <p>Current Price: ₹{current_price:.2f} Lakhs</p>
                        </div>
                        """,
                        unsafe_allow_html=True
                    )

                # ROI Calculation
                roi = ((reg_pred - current_price) / current_price) * 100
                total_gain = reg_pred - current_price

                st.markdown("---")
                st.header("💹 Investment Analysis")

                col_x, col_y, col_z = st.columns(3)
                col_x.metric("Expected ROI", f"{roi:.2f}%", f"{roi:.2f}%")
                col_y.metric("Total Gain", f"₹{total_gain:.2f} Lakhs", f"₹{total_gain:.2f}L")
                col_z.metric("Annual Return", f"{roi/5:.2f}%/year", f"{roi/5:.2f}%")

            except Exception as e:
                st.error(f"Prediction error: {e}")

    with tab2:
        st.header("📊 Market Insights")

        # City-wise statistics
        st.subheader("🏙️ Average Prices by City (Top 10)")
        city_avg = reference_data.groupby(\\'City\\')[\\'Price_in_Lakhs\\'].mean().sort_values(ascending=False).head(10)
        st.bar_chart(city_avg)

        # Property type distribution
        st.subheader("🏠 Property Type Distribution")
        prop_dist = reference_data[\\'Property_Type\\'].value_counts()
        st.bar_chart(prop_dist)

    with tab3:
        st.header("ℹ️ About This Application")
        st.markdown("""
        ### Real Estate Investment Advisor

        This AI-powered application helps investors make informed decisions about real estate investments in India.

        **Features:**
        - 🎯 **Classification**: Determines if a property is a good investment
        - 📈 **Regression**: Predicts property price after 5 years
        - 💹 **ROI Calculator**: Shows expected returns

        **Models Used:**
        - XGBoost Classifier (tuned)
        - XGBoost Regressor (tuned)

        **Dataset:**
        - 247,000+ Indian properties
        - 20+ features including location, amenities, and market data

        **Tech Stack:**
        - Python, Scikit-learn, XGBoost
        - Streamlit, MLflow
        - Pandas, NumPy

        ---
        **Created by:** Your Name
        **Project:** Real Estate Investment Advisor
        """)

if __name__ == "__main__":
    main()
'''

# Save app.py
app_path = f'{app_dir}/app.py'
with open(app_path, 'w') as f:
    f.write(app_code)

print(f"✅ Main app created: {app_path}")


✅ Main app created: /content/drive/MyDrive/assignment-3/streamlit_app/app.py
